In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime,timedelta
import json
import re

In [2]:
#根据log_index，增量读取log内容
def load_log(file_name, log_index):
    log_value_list = list()
    date_list = list()
    status_list = list()
    try:
        with open(file_name,'rb') as f:
            for index,content in enumerate(f):
                if index == log_index:
                    log_data = json.loads(content.decode('utf8'))
                    log_data['@timestamp'] = str(datetime.strptime(log_data['@timestamp'][0:19],'%Y-%m-%dT%H:%M:%S'))
                    log_value_list.append(log_data)
                    date_list.append(log_data['@timestamp'])
                    log_index += 1
                    if log_data['status'] not in status_list:
                        status_list.append(log_data['status'])
    except Exception as e:
        print(e)
    #将文本内容转化为pandas的dataframe
    #temp_log = pd.DataFrame(log_value_list)
    print(status_list)
    return  log_value_list,log_index

In [3]:
time_step = 60 #每x秒读取一次log文件
time_windows = 120 #秒,时间窗口
ip_dict = dict() #ip与对应操作在矩阵的下标
ip_reserve_list = list() #操作矩阵下标对应的ip
all_matrix = np.empty([0, 6],dtype=int)  #操作矩阵
all_log = pd.DataFrame() #所有的日志数据
temp_log_data = pd.DataFrame() #某时间范围的日志数据
all_url_list = list()
time_index = '2017-07-10 06:25:08' #记录当前的时间
log_index = 0 #记录读到第几行了
ip_behavior_dict = dict() #记录每个ip每分钟请求的页面url

increment_log_data, log_index = load_log('0713_access_sdsl_shunnengnet_com_json.log', 0)

['200', '302', '304', '403', '499', '404', '503', '408', '400', '504']


In [4]:
#截取url，保留其访问传参方式（POST/GET/PUT/DELETE）,保留url的头与尾
def deal_url(url):
    url_str = url.replace(' HTTP/1.1','')
    if url_str.find(' ') > -1:
        url_type = url_str[0 : url_str.find(' ')+1]
    if url_str.find('/') > -1 and url_str.find('?') > -1:
        url_str = url_type + url_str[url_str.find('/')+1 : url_str.find('?')]
    if url_str.find('/') > -1 and url_str.rfind('/') > -1:
        url_str = url_str[0 : url_str.find('/')] + '/.../' + url_str[url_str.rfind('/')+1 : ]
    return url_str

In [5]:
#根据日志内容，创建一个信息的op_detail
def create_op_detail(row_log):
    op_detail = dict()
    op_detail['start_time'] = row_log['@timestamp']
    op_detail['end_time'] = row_log['@timestamp']
    op_detail['urls'] = list()
    op_detail['urls'].append(deal_url(row_log['request']))
    return op_detail

In [6]:
#传入开始结束时间的字符串(yyyy-MM-dd HH:mm:ss)，计算时间差并返回时差(秒)
def cacl_time_difference(start_time_str, end_time_str):
    start_time = datetime.strptime(start_time_str,'%Y-%m-%d %H:%M:%S')
    end_time = datetime.strptime(end_time_str,'%Y-%m-%d %H:%M:%S')
    return int((end_time - start_time).seconds)

In [7]:
#读取log，生成携带用户行为的复合型字典
def create_user_behavior_dict(increment_log_data, user_behavior_dict):
    
    for x in increment_log_data:
        if x['sip'] != '':
            
            if x['sip'] not in user_behavior_dict.keys():
                user_behavior_dict[x['sip']] = list()

            if len(user_behavior_dict[x['sip']]) == 0: #opreation内没有op_detail,则创建op_detail
                user_behavior_dict[x['sip']].append(create_op_detail(x))

            if len(user_behavior_dict[x['sip']]) > 0: #opreation内已存在op_detail，此时需要循环该list的所有dict，比较时间差，如果两者时间相差<120s，更新结束时间，添加url；否则继续遍历比较直至找不到合适的区间为止，若找不到合适区间，新建
                new_opreation = 1
                length = len(user_behavior_dict[x['sip']])
                i = 0
                for op_detail in user_behavior_dict[x['sip']]:
                    time_difference = cacl_time_difference(op_detail['end_time'], x['@timestamp'])
#                     if x['sip'] == '223.96.152.23':
#                         print(op_detail)
                    if time_difference < 120:
                        op_detail['end_time'] = x['@timestamp']
                        op_detail['urls'].append(deal_url(x['request']))
                        new_opreation = 0
                        break
                    if new_opreation == 1 and i+1 == length:
                        user_behavior_dict[x['sip']].append(create_op_detail(x))
                        break
                        
                    i += 1
                    
                
    return user_behavior_dict
        
        

In [8]:
user_behavior_dict = dict()

In [9]:
start_time = datetime.now()
user_behavior_dict = create_user_behavior_dict(increment_log_data, user_behavior_dict)
end_time = datetime.now()
print((end_time - start_time).seconds)

24


In [10]:
#定义模板list
templates = list()
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do']) #普通专家预约
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../expertNotice.do','POST /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#知名专家预约
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../specialClinic.do','POST /.../getDepts.do','GET /common/.../specialSchedule.do','POST /.../specialSchedule.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#特色门诊预约
templates.append(['GET /common/.../patientinfo.do','GET /common/.../reserveList.do','POST /.../getReserveList.do','GET /common/.../reserveDetail.do','POST /.../cancelReserve.do'])#取消预约
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /.../patientinfo.do','GET /common/.../addPatient.do','POST /.../checkIdCard.do','POST /.../create.do'])#新增就诊人
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','POST /.../setDefault.do',''])#设置默认就诊人
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /common/.../bind.do','POST /.../bindHosCard.do'])#绑卡
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /.../deletePatient.do'])#删除就诊人
templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportList.do','GET /common/.../reportDetail.do','GET /.../getReportDeatil.do'])#门诊报告
templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportInList.do','GET /.../getReportDeatil.do'])#住院报告
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../balance.do','GET /.../patientBalance.do'])#门诊预交金
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../hosBalance.do','POST /.../inBalance.do'])#住院预交金
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../outpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#门诊费用
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../inpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#住院费用
templates.append(['GET /.../init.do','GET /common/.../reserveQueueList.do','POST /.../getQueueList.do'])#候诊队列
templates.append(['POST coreServlet'])#处理微信发送来的数据
templates.append(['GET common/.../arrangementList.do', 'GET /.../404.do','POST /.../getTimeList.do'])#异常模板1
templates.append(['GET common/.../scheduleByDept.do','POST /.../scheduleByDept.do','POST /.../getTimeList.do','GET common/.../arrangementList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do','POST /.../getTimeList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板2
templates.append(['GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板3

In [64]:
probability_dict = dict()

for x in user_behavior_dict.keys():
    for op_detail in user_behavior_dict[x]:
        matched_operations = np.zeros(len(templates),dtype='float32')
        for idx,template in enumerate(templates):
            n_template = len(template)
            count = 0
            probability_count = 0
            for str in template:
                if str in op_detail['urls']:
                    probability_count += 1
            for url in op_detail['urls']:
                if url in template:
                    count += 1
            #print(probability_count / n_template,count / n_template)
            matched_operations[idx] = (probability_count / n_template) * (count / n_template)
        if x not in probability_dict.keys():
            probability_dict[x] = matched_operations
        else:
            probability_dict[x] = np.vstack((probability_dict[x],matched_operations))
        
        

In [65]:
len(probability_dict['223.104.186.241'])

6

In [66]:
len(user_behavior_dict['223.104.186.241'])

6

In [67]:
np.vstack((matched_operations,matched_operations))

array([[ 0.02777778,  0.02366864,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.04938272,  0.        ],
       [ 0.02777778,  0.02366864,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.04938272,  0.        ]], dtype=float32)

In [71]:
user_behavior_dict['43.224.213.102']

[{'end_time': '2017-07-13 12:48:53',
  'start_time': '2017-07-13 12:47:46',
  'urls': ['GET common/.../notice.do',
   'GET common/.../notice.do',
   'POST common/.../chooseHospital.do',
   'GET common/.../chooseExpert.do',
   'GET common/.../deptList.do',
   'POST /.../getDept.do',
   'GET common/.../scheduleByDept.do',
   'POST /.../scheduleByDept.do',
   'GET common/.../arrangementList.do',
   'GET common/.../arrangementList.do',
   'GET common/.../arrangementList.do',
   'POST /.../getTimeList.do',
   'POST /.../scheduleByDept.do',
   'GET common/.../arrangementList.do',
   'POST /.../getTimeList.do',
   'POST /.../scheduleByDept.do',
   'POST /.../getDept.do',
   'GET common/.../patientinfo.do',
   'GET common/.../reserveList.do',
   'POST /.../getReserveList.do']},
 {'end_time': '2017-07-13 15:30:39',
  'start_time': '2017-07-13 15:29:54',
  'urls': ['GET common/.../patientinfo.do',
   'GET common/.../reserveList.do',
   'POST /.../getReserveList.do',
   'GET common/.../guide.do',

In [ ]:
probability_dict['43.224.213.102'][index,16:].sum()>5

In [50]:
def trans_opreations_vector(probability_dict, templates_length):
    operation_vector_dict = dict()
    operation_vector_sum_dict = dict()
    for ip in probability_dict.keys():
        if ip not in operation_vector_dict.keys():
            operation_vector_dict[ip] = list()
        for column_vector in probability_dict[ip]:
            operation_vector = np.zeros(templates_length, dtype='float32')
            i = 0
            for column_vector_item in column_vector:
                operation_vector[i] = column_vector_item[0]
                i += 1
            operation_vector_dict[ip].append(operation_vector)
    
    for ip in operation_vector_dict.keys():
        if ip not in operation_vector_sum_dict.keys():
            operation_vector_sum_dict[ip] = np.zeros(templates_length, dtype='float32')
        for x in operation_vector_dict[ip]:
            operation_vector_sum_dict[ip] += x
            
    return operation_vector_sum_dict
            
        

In [51]:
opreations_vectors = trans_opreations_vector(probability_dict, len(templates))

IndexError: invalid index to scalar variable.

In [64]:
len(opreations_vectors['223.104.186.241'])

19

In [65]:
opreations_vectors['223.104.186.241']

array([ 0.59027779,  0.5029586 ,  0.08333334,  0.08      ,  0.5       ,
        0.02777778,  0.11111111,  0.        ,  0.02777778,  0.04      ,
        0.0625    ,  0.0625    ,  0.02777778,  0.02777778,  0.11111111,
        0.        ,  1.        ,  1.14814818,  0.3125    ], dtype=float32)

In [66]:
frequency_dict = dict()
for x in increment_log_data:
    if x['sip'] != '':
        if x['sip'] not in frequency_dict.keys():
            frequency_dict[x['sip']] = 0
        frequency_dict[x['sip']] += 1

In [67]:
print(sorted(frequency_dict.items(), key=lambda d: d[1]))

[('218.57.245.174', 1), ('219.72.200.201', 1), ('58.56.83.186', 1), ('60.208.219.74', 1), ('223.98.173.10', 1), ('124.134.25.18', 1), ('113.129.185.247', 1), ('58.58.52.52', 1), ('112.224.20.26', 1), ('113.128.89.2', 1), ('39.71.134.40', 1), ('223.98.46.74', 1), ('122.5.121.26', 1), ('117.185.27.98', 1), ('113.128.60.12', 1), ('113.128.96.130', 1), ('111.15.41.154', 1), ('115.239.212.139', 1), ('140.207.54.140', 1), ('111.17.78.177', 1), ('112.246.120.252', 1), ('112.232.121.233', 1), ('112.224.2.116', 1), ('219.146.71.172', 1), ('183.232.118.15', 1), ('39.64.0.222', 1), ('45.124.46.51', 1), ('111.34.23.46', 1), ('61.179.23.248', 1), ('112.224.20.118', 1), ('111.30.137.188', 1), ('111.40.197.20', 1), ('123.134.12.216', 1), ('111.30.131.238', 1), ('113.129.24.3', 1), ('112.224.19.254', 1), ('112.241.119.210', 1), ('113.129.6.200', 1), ('117.136.94.54', 1), ('112.224.69.198', 1), ('113.129.224.221', 1), ('111.14.185.228', 1), ('218.56.223.206', 1), ('112.230.72.252', 1), ('123.171.135.20

In [71]:
user_behavior_dict['27.204.15.164']

[{'end_time': '2017-07-13 20:28:14',
  'start_time': '2017-07-13 19:18:50',
  'urls': ['POST /.../getTimeList.do',
   'POST /.../getTimeList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'POST /.../getTimeList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'POST /.../getTimeList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'POST /.../getTimeList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'POST /.../getTimeList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',
   'GET common/.../arrangementList.do',
   'GET /.../404.do',


In [72]:
opreations_vectors['27.204.19.242']

array([  1.78368057e+02,   1.51982254e+02,   1.06791664e+02,
         2.39999995e-01,   7.77777791e-01,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   1.71033337e+03,   5.71444458e+02,
         4.27416656e+02], dtype=float32)

In [89]:
vector = np.zeros((1,len(templates)), dtype='float32')
for x in opreations_vectors.keys():
    #if len(opreations_vectors[x]) != 19:
        #print(opreations_vectors[x])
    vector = np.concatenate((vector,[opreations_vectors[x]]), axis=0)

In [95]:
vector

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.90277791e-01,   5.02958596e-01,   8.33333358e-02, ...,
          1.00000000e+00,   1.14814818e+00,   3.12500000e-01],
       [  8.33333358e-02,   7.10059181e-02,   6.94444450e-03, ...,
          6.66666687e-01,   5.55555582e-01,   2.50000000e-01],
       ..., 
       [  1.68750000e+00,   1.43786979e+00,   6.94444450e-03, ...,
          4.44444448e-01,   9.22222233e+00,   1.66666672e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.77777780e-02,   2.36686394e-02,   0.00000000e+00, ...,
          0.00000000e+00,   4.93827164e-02,   0.00000000e+00]], dtype=float32)

In [99]:
len(vector)

7375

In [104]:
vector[:, 18] > 1.0

array([False, False, False, ..., False, False, False], dtype=bool)

In [103]:
vector.shape

(7375, 19)